# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse          Shipments  \
0  May 23 2022 12:41PM  237872        19        ADV80002817   
1  May 23 2022 12:41PM  237872        19        ADV80002818   
2  May 23 2022 12:41PM  237872        19        ADV80002819   
3  May 23 2022 12:41PM  237872        19        ADV80002820   
4  May 23 2022 12:41PM  237872        19        ADV80002821   
5  May 23 2022 12:41PM  237872        19        ADV80002822   
6  May 23 2022 12:41PM  237872        19        ADV80002823   
7  May 23 2022 12:39PM  237871        10            8044754   
8  May 23 2022 12:36PM  237870        10  SNAP4500026139-BO   
9  May 23 2022 12:36PM  237870        10   SNAP-29472398-BO   

                       Customer ShipmentStatus  
0           AdvaGen Pharma, Ltd       Released  
1           AdvaGen Pharma, Ltd       Released  
2           AdvaGen Pharma, Ltd       Released  
3           AdvaGen Pharma, Ltd       Released  
4           AdvaGen Pharma, Ltd       Released  
5           AdvaGen Pharma, Ltd       Released  
6           AdvaGen Pharma, Ltd       Released  
7           MedStone Pharma LLC       Released  
8  Snap Medical Industries, LLC       Released  
9  Snap Medical Industries, LLC       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
36  237867       Released          1
37  237869       Released          8
38  237870       Released          8
39  237871       Released          1
40  237872       Released          7

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
237867                NaN        NaN       1.0
237869                NaN        NaN       8.0
237870                NaN        NaN       8.0
237871                NaN        NaN       1.0
237872                NaN        NaN       7.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
237673                0.0        0.0       1.0
237794                0.0        7.0      19.0
237795                0.0        0.0       1.0
237796                0.0        0.0       1.0
237804                4.0       10.0       5.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
237673                  0          0         1
237794                  0          7        19
237795                  0          0         1
237796                  0          0         1
237804                  4         10         5

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               237673          0          0         1
1               237794          0          7        19
2               237795          0          0         1
3               237796          0          0         1
4               237804          4         10         5

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               237673                            1
1               237794                   7       19
2               237795                            1
3               237796                            1
4               237804         4        10        5

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                      Customer
0   May 23 2022 12:41PM  237872        19           AdvaGen Pharma, Ltd
7   May 23 2022 12:39PM  237871        10           MedStone Pharma LLC
8   May 23 2022 12:36PM  237870        10  Snap Medical Industries, LLC
16  May 23 2022 12:34PM  237869        10             ISDIN Corporation
24  May 23 2022 12:25PM  237867        22             NBTY Global, Inc.
25  May 23 2022 12:24PM  237866        22             NBTY Global, Inc.
26  May 23 2022 12:23PM  237863        10                       Bio-PRF
43  May 23 2022 12:20PM  237673        18                     Hush Hush
44  May 23 2022 11:57AM  237861        20      ACI Healthcare USA, Inc.
67  May 23 2022 11:51AM  237860        12             NBTY Global, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                      Customer  \
0  May 23 2022 12:41PM  237872        19           AdvaGen Pharma, Ltd   
1  May 23 2022 12:39PM  237871        10           MedStone Pharma LLC   
2  May 23 2022 12:36PM  237870        10  Snap Medical Industries, LLC   
3  May 23 2022 12:34PM  237869        10             ISDIN Corporation   
4  May 23 2022 12:25PM  237867        22             NBTY Global, Inc.   
5  May 23 2022 12:24PM  237866        22             NBTY Global, Inc.   
6  May 23 2022 12:23PM  237863        10                       Bio-PRF   
7  May 23 2022 12:20PM  237673        18                     Hush Hush   
8  May 23 2022 11:57AM  237861        20      ACI Healthcare USA, Inc.   
9  May 23 2022 11:51AM  237860        12             NBTY Global, Inc.   

  Completed Executing Released  
0                            7  
1                            1  
2                            8  
3                            8  
4                            1  
5                            1  
6                           17  
7                            1  
8                           23  
9                            2

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                      Customer  \
0  May 23 2022 12:41PM  237872        19           AdvaGen Pharma, Ltd   
1  May 23 2022 12:39PM  237871        10           MedStone Pharma LLC   
2  May 23 2022 12:36PM  237870        10  Snap Medical Industries, LLC   
3  May 23 2022 12:34PM  237869        10             ISDIN Corporation   
4  May 23 2022 12:25PM  237867        22             NBTY Global, Inc.   
5  May 23 2022 12:24PM  237866        22             NBTY Global, Inc.   
6  May 23 2022 12:23PM  237863        10                       Bio-PRF   
7  May 23 2022 12:20PM  237673        18                     Hush Hush   
8  May 23 2022 11:57AM  237861        20      ACI Healthcare USA, Inc.   
9  May 23 2022 11:51AM  237860        12             NBTY Global, Inc.   

  Released Executing Completed  
0        7                      
1        1                      
2        8                      
3        8                      
4        1                      
5        1                      
6       17                      
7        1                      
8       23                      
9        2

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  May 23 2022 12:41PM  237872        19           AdvaGen Pharma, Ltd   
1  May 23 2022 12:39PM  237871        10           MedStone Pharma LLC   
2  May 23 2022 12:36PM  237870        10  Snap Medical Industries, LLC   
3  May 23 2022 12:34PM  237869        10             ISDIN Corporation   
4  May 23 2022 12:25PM  237867        22             NBTY Global, Inc.   

  Released Executing Completed  
0        7                      
1        1                      
2        8                      
3        8                      
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                      Customer  \
0  May 23 2022 12:41PM  237872        19           AdvaGen Pharma, Ltd   
1  May 23 2022 12:39PM  237871        10           MedStone Pharma LLC   
2  May 23 2022 12:36PM  237870        10  Snap Medical Industries, LLC   
3  May 23 2022 12:34PM  237869        10             ISDIN Corporation   
4  May 23 2022 12:25PM  237867        22             NBTY Global, Inc.   

   Released  Executing  Completed  
0       7.0        NaN        NaN  
1       1.0        NaN        NaN  
2       8.0        NaN        NaN  
3       8.0        NaN        NaN  
4       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  May 23 2022 12:41PM  237872        19           AdvaGen Pharma, Ltd   
1  May 23 2022 12:39PM  237871        10           MedStone Pharma LLC   
2  May 23 2022 12:36PM  237870        10  Snap Medical Industries, LLC   
3  May 23 2022 12:34PM  237869        10             ISDIN Corporation   
4  May 23 2022 12:25PM  237867        22             NBTY Global, Inc.   

   Released  Executing  Completed  
0       7.0        0.0        0.0  
1       1.0        0.0        0.0  
2       8.0        0.0        0.0  
3       8.0        0.0        0.0  
4       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2854082     313.0       19.0        4.0
12          713554       3.0        1.0        0.0
15          475600      25.0       13.0        2.0
18         1664541       7.0        0.0        0.0
19          713571       8.0        1.0        0.0
20          951413      63.0       19.0        1.0
22          475733       2.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2854082     313.0       19.0        4.0
1        12   713554       3.0        1.0        0.0
2        15   475600      25.0       13.0        2.0
3        18  1664541       7.0        0.0        0.0
4        19   713571       8.0        1.0        0.0
5        20   951413      63.0       19.0        1.0
6        22   475733       2.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     313.0       19.0        4.0
1        12       3.0        1.0        0.0
2        15      25.0       13.0        2.0
3        18       7.0        0.0        0.0
4        19       8.0        1.0        0.0
5        20      63.0       19.0        1.0
6        22       2.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  313.0
1        12  Released    3.0
2        15  Released   25.0
3        18  Released    7.0
4        19  Released    8.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   12    15   18   19    20   22
Status                                          
Completed    4.0  0.0   2.0  0.0  0.0   1.0  0.0
Executing   19.0  1.0  13.0  0.0  1.0  19.0  0.0
Released   313.0  3.0  25.0  7.0  8.0  63.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   12    15   18   19    20   22
0          Completed    4.0  0.0   2.0  0.0  0.0   1.0  0.0
1          Executing   19.0  1.0  13.0  0.0  1.0  19.0  0.0
2           Released  313.0  3.0  25.0  7.0  8.0  63.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   12    15   18   19    20   22
0  Completed    4.0  0.0   2.0  0.0  0.0   1.0  0.0
1  Executing   19.0  1.0  13.0  0.0  1.0  19.0  0.0
2   Released  313.0  3.0  25.0  7.0  8.0  63.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()